Dans ce billet, nous allons extraire les données disponibles sur les scrutins de l'assemblée nationale afin de déterminer quels jours de la semaine sont propice au débat dans l'assemblée et quels jours ne le sont pas.

# Récupération des informations 

Les données qui m'intéressent sont sur la page suivante : [http://www2.assemblee-nationale.fr/scrutins/liste/%28legislature%29/14](http://www2.assemblee-nationale.fr/scrutins/liste/%28legislature%29/14).

Téléchargeons le contenu de cette page.

In [2]:
import requests

In [5]:
r = requests.get("http://www2.assemblee-nationale.fr/scrutins/liste/%28legislature%29/14")
r

<Response [200]>

Maintenant, passons la page dans le parser BeautifulSoup :

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')

Les éléments qui nous intéressent sont les suivants : 

- numéro du scrutin
- date du scrutin
- objet du scrutin
- résultats : pour, contre, abstention volontaire

Ces éléments sont dans un grand tableau, que nous allons essayer d'extraire :

In [8]:
elems = soup.select('tbody tr')

In [9]:
len(elems)

100

Chaque élément précédent représente une ligne du tableau. La première ligne du tableau contient par exemple les informations suivantes :

In [10]:
elems[0]

<tr>
<td class="denom">1237*</td>
<td>10/02/2016</td>
<td class="desc">l'ensemble du projet de loi constitutionnelle de protection de la Nation (première lecture). [<a href="http://www.assemblee-nationale.fr/14/dossiers/protection_nation.asp">dossier</a>] [<a href="/scrutins/detail/(legislature)/14/(num)/1237">analyse du scrutin</a>]</td>
<td class="pour">317</td>
<td class="contre SSO">199</td>
<td class="abs SSO">51</td>
</tr>

Ecrivons une fonction qui extrait les informations d'une seule ligne.

In [63]:
def extract_information(line):
    """Extrait les informations d'une ligne du tableau des scrutins."""
    elems = line.find_all('td')
    # numéro de scrutin
    num = elems[0].text
    # date
    date = elems[1].text
    # objet
    obj = elems[2].text
    # pour
    pour = int(elems[3].text)
    # contre
    contre = int(elems[4].text)
    # abstention
    abstention = int(elems[5].text)
    return num, date, obj, pour, contre, abstention

On peut tester cette routine sur la première ligne :

In [64]:
extract_information(elems[0])

('1237*',
 '10/02/2016',
 "l'ensemble du projet de loi constitutionnelle de protection de la Nation (première lecture). [dossier] [analyse du scrutin]",
 317,
 199,
 51)

Répétons cette extraction pour chaque ligne :

In [65]:
data = [extract_information(line) for line in elems]

Comme on peut le voir, cela marche plutôt bien. Nous allons maintenant traiter ces informations avec `pandas`.

# Utilisation de pandas 

Nous pouvons directement importer les données précédentes dans un objet dataframe, qui va nous simplifier les démarches :

In [66]:
import pandas as pd

In [67]:
df = pd.DataFrame(data, columns=('numero', 'date', 'objet', 'pour', 'contre', 'abstention'))
df.head(10)

,numero,date,objet,pour,contre,abstention
0,1237*,10/02/2016,l'ensemble du projet de loi constitutionnelle ...,317,199,51
1,1236,09/02/2016,l'ensemble de l'article premier du projet de l...,116,19,11
2,1235,09/02/2016,l'amendement n° 1 du Gouvernement à l'article ...,132,7,7
3,1234,09/02/2016,l'amendement n° 2 du Gouvernement à l'article ...,104,34,7
4,1233,09/02/2016,l'amendement n° 234 de M. Hamon après l'articl...,61,97,3
5,1232,09/02/2016,l'amendement n° 168 de M. Hamon après l'articl...,67,103,2
6,1231,09/02/2016,l'article 2 du projet de loi constitutionnelle...,162,148,22
7,1230,09/02/2016,le sous-amendement n° 271 de Mme Duflot et les...,59,185,25
8,1229,09/02/2016,le sous-amendement n° 250 de M. Goldberg et le...,77,174,32
9,1228,09/02/2016,l'amendement n° 50 de M. Schwartzenberg à l'ar...,64,180,41


A partir de ce tableau, nous pouvons calculer le nombre de d'absents, car il y a 577 députés à l'assemblée nationale.

In [68]:
df['absents'] = 577 - (df['pour'] + df['contre'] + df['abstention'])

In [69]:
df.head(10)

,numero,date,objet,pour,contre,abstention,absents
0,1237*,10/02/2016,l'ensemble du projet de loi constitutionnelle ...,317,199,51,10
1,1236,09/02/2016,l'ensemble de l'article premier du projet de l...,116,19,11,431
2,1235,09/02/2016,l'amendement n° 1 du Gouvernement à l'article ...,132,7,7,431
3,1234,09/02/2016,l'amendement n° 2 du Gouvernement à l'article ...,104,34,7,432
4,1233,09/02/2016,l'amendement n° 234 de M. Hamon après l'articl...,61,97,3,416
5,1232,09/02/2016,l'amendement n° 168 de M. Hamon après l'articl...,67,103,2,405
6,1231,09/02/2016,l'article 2 du projet de loi constitutionnelle...,162,148,22,245
7,1230,09/02/2016,le sous-amendement n° 271 de Mme Duflot et les...,59,185,25,308
8,1229,09/02/2016,le sous-amendement n° 250 de M. Goldberg et le...,77,174,32,294
9,1228,09/02/2016,l'amendement n° 50 de M. Schwartzenberg à l'ar...,64,180,41,292
